In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from arm_pkg import arm

import pybullet as b
import time
import pybullet_data
import numpy as np
import eigen as e

DT = 0.001 #s
MOVEMENT_TIME = 10000 #ms
URDF_PATH = "../urdf/four_dof.urdf"

def inv_spd_sr(mat, k=1e-4):
    '''Converts an inverse into an SVD problem with singularity robust inverse
    @param mat Matrix to be inversed
    @param tolerance Tolerance to the minimum singular value that can go through reciprocal operation
    @return Homogeneous transform matrix
    '''
    mat_np = np.array(mat)
    reg = k * np.identity(np.size(mat_np,0))
    
    return e.MatrixXd(mat_np.transpose().dot(np.linalg.inv(mat_np.dot(mat_np.transpose()) + reg)))

# setup simulation environment
physicsClient = b.connect(b.GUI)
b.setAdditionalSearchPath(pybullet_data.getDataPath())
b.setGravity(0,0,-9.81)
b.setRealTimeSimulation(0)
b.setTimeStep(DT)
a = arm.Arm(URDF_PATH)

# import robot
planeId = b.loadURDF("plane.urdf")
startPos = [0,0,1]
startOrientation = b.getQuaternionFromEuler([0,0,0])
loadFlag = b.URDF_USE_INERTIA_FROM_FILE | b.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS
robotId = b.loadURDF(URDF_PATH,startPos, startOrientation, flags=loadFlag)
nDof = b.getNumJoints(robotId)
jointsId = range(nDof)

# setup configuration
S = np.array([1,0,1,0,0,0])
ofs_rot = np.identity(3)
ofs_pos = np.array([1,0,0])
b.setJointMotorControlArray(robotId, jointsId, b.VELOCITY_CONTROL, forces=[0] * nDof)
q_home = range(nDof)
for i in jointsId:
    b.resetJointState(robotId, i, targetValue=q_home[i])
    
# simulate torque control
for i in range(100000):
    joint_states = b.getJointStates(robotId, jointsId)
    
    # read state feedback
    a.set_q([joint_states[i][0] for i in jointsId])
    a.set_dq([joint_states[i][1] for i in jointsId])
    
    # forward kine
    a.forward_kine()
    
    # get pose
    T1 = a.get_body_trans(ofs_rot, ofs_pos, "link_4")
    T2 = a.get_body_trans(ofs_rot, ofs_pos, "link_2")
    
    # get jacobian
#     a.set_jacobian(ofs_pos, "link_4")
#     J1 = e.MatrixXd(6,nDof)
#     dJ1 = e.MatrixXd(6,nDof)
    a.get_jacobian_and_deriv(ofs_pos, "link_4")
    
    jac = a.get_jacobian()
    j = jac.jacobian(a.get_mb(), a.get_mbc())